In [1]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import cifar10

import glob
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#VGG16はkeras標準でセットされている
model = VGG16()
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# 論文実装

In [3]:
#inputは224×224×3
#カーネルは3×3
#ReLU
#モデルはA~Eまである(VGG_model_image.png)

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [5]:
#modelA 11層(8層畳み込み、3層全結合)

input_shape = (224,224,3)
# print('入力シェープ = ', input_shape)

num_classes = 1000

#VGG A
model_A = Sequential()

# Conv1
model_A.add(Conv2D(64, (3,3),activation='relu', padding='same',
                 input_shape=(224,224,3)))
# Pool1
model_A.add(MaxPooling2D(pool_size=(2, 2)))

# Conv2
model_A.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Pool2
model_A.add(MaxPooling2D(pool_size=(2, 2)))

# Conv3
model_A.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_A.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Pool3
model_A.add(MaxPooling2D(pool_size=(2, 2)))

# Conv4
model_A.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_A.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Pool4
model_A.add(MaxPooling2D(pool_size=(2, 2)))

# Conv5
model_A.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv5
model_A.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Pool5
model_A.add(MaxPooling2D(pool_size=(2, 2)))

model_A.add(Flatten())
# fc6
model_A.add(Dense(4096, activation='relu'))
# fc7
model_A.add(Dense(4096, activation='relu'))
# fc8
model_A.add(Dense(num_classes, activation='softmax'))

model_A.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 256)       590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 256)       0

In [6]:
#modelB 13層(10層畳み込み、3層全結合)

input_shape = (224,224,3)
# print('入力シェープ = ', input_shape)

num_classes = 1000

#VGG B
model_B = Sequential()

# Conv1
model_B.add(Conv2D(64, (3,3),activation='relu', padding='same',
                 input_shape=(224,224,3)))
# Conv1
model_B.add(Conv2D(64, (3,3),activation='relu', padding='same'))
# Pool1
model_B.add(MaxPooling2D(pool_size=(2, 2)))

# Conv2
model_B.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Conv2
model_B.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Pool2
model_B.add(MaxPooling2D(pool_size=(2, 2)))

# Conv3
model_B.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_B.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Pool3
model_B.add(MaxPooling2D(pool_size=(2, 2)))

# Conv4
model_B.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_B.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Pool4
model_B.add(MaxPooling2D(pool_size=(2, 2)))

# Conv5
model_B.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv5
model_B.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Pool5
model_B.add(MaxPooling2D(pool_size=(2, 2)))

model_B.add(Flatten())
# fc6
model_B.add(Dense(4096, activation='relu'))
# fc7
model_B.add(Dense(4096, activation='relu'))
# fc8
model_B.add(Dense(num_classes, activation='softmax'))

model_B.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 56, 56, 256)      

In [7]:
#modelC 16層(13層畳み込み、3層全結合、畳み込みに1×1カーネル)

input_shape = (224,224,3)
# print('入力シェープ = ', input_shape)

num_classes = 1000

#VGG C
model_C = Sequential()

# Conv1
model_C.add(Conv2D(64, (3,3),activation='relu', padding='same',
                 input_shape=(224,224,3)))
# Conv1
model_C.add(Conv2D(64, (3,3),activation='relu', padding='same'))
# Pool1
model_C.add(MaxPooling2D(pool_size=(2, 2)))

# Conv2
model_C.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Conv2
model_C.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Pool2
model_C.add(MaxPooling2D(pool_size=(2, 2)))

# Conv3
model_C.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_C.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_C.add(Conv2D(256, (1,1),activation='relu', padding='same'))
# Pool3
model_C.add(MaxPooling2D(pool_size=(2, 2)))

# Conv4
model_C.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_C.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_C.add(Conv2D(512, (1,1),activation='relu', padding='same'))
# Pool4
model_C.add(MaxPooling2D(pool_size=(2, 2)))

# Conv5
model_C.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv5
model_C.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv5
model_C.add(Conv2D(512, (1,1),activation='relu', padding='same'))
# Pool5
model_C.add(MaxPooling2D(pool_size=(2, 2)))

model_C.add(Flatten())
# fc6
model_C.add(Dense(4096, activation='relu'))
# fc7
model_C.add(Dense(4096, activation='relu'))
# fc8
model_C.add(Dense(num_classes, activation='softmax'))

model_C.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 56, 56, 256)      

In [8]:
#modelC 16層(13層畳み込み、3層全結合)

input_shape = (224,224,3)
# print('入力シェープ = ', input_shape)

num_classes = 1000

#VGG D
model_D = Sequential()

# Conv1
model_D.add(Conv2D(64, (3,3),activation='relu', padding='same',
                 input_shape=(224,224,3)))
# Conv1
model_D.add(Conv2D(64, (3,3),activation='relu', padding='same'))
# Pool1
model_D.add(MaxPooling2D(pool_size=(2, 2)))

# Conv2
model_D.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Conv2
model_D.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Pool2
model_D.add(MaxPooling2D(pool_size=(2, 2)))

# Conv3
model_D.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_D.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_D.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Pool3
model_D.add(MaxPooling2D(pool_size=(2, 2)))

# Conv4
model_D.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_D.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_D.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Pool4
model_D.add(MaxPooling2D(pool_size=(2, 2)))

# Conv5
model_D.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv5
model_D.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv5
model_D.add(Conv2D(512, (1,1),activation='relu', padding='same'))
# Pool5
model_D.add(MaxPooling2D(pool_size=(2, 2)))

model_D.add(Flatten())
# fc6
model_D.add(Dense(4096, activation='relu'))
# fc7
model_D.add(Dense(4096, activation='relu'))
# fc8
model_D.add(Dense(num_classes, activation='softmax'))

model_D.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 56, 56, 256)      

In [9]:
#modelC 19層(16層畳み込み、3層全結合)

input_shape = (224,224,3)
# print('入力シェープ = ', input_shape)

num_classes = 1000

#VGG E
model_E = Sequential()

# Conv1
model_E.add(Conv2D(64, (3,3),activation='relu', padding='same',
                 input_shape=(224,224,3)))
# Conv1
model_E.add(Conv2D(64, (3,3),activation='relu', padding='same'))
# Pool1
model_E.add(MaxPooling2D(pool_size=(2, 2)))

# Conv2
model_E.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Conv2
model_E.add(Conv2D(128, (3,3),activation='relu', padding='same'))
# Pool2
model_E.add(MaxPooling2D(pool_size=(2, 2)))

# Conv3
model_E.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_E.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_E.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Conv3
model_E.add(Conv2D(256, (3,3),activation='relu', padding='same'))
# Pool3
model_E.add(MaxPooling2D(pool_size=(2, 2)))

# Conv4
model_E.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_E.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_E.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv4
model_E.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Pool4
model_E.add(MaxPooling2D(pool_size=(2, 2)))

# Conv5
model_E.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv5
model_E.add(Conv2D(512, (3,3),activation='relu', padding='same'))
# Conv5
model_E.add(Conv2D(512, (1,1),activation='relu', padding='same'))
# Conv5
model_E.add(Conv2D(512, (1,1),activation='relu', padding='same'))
# Pool5
model_E.add(MaxPooling2D(pool_size=(2, 2)))

model_E.add(Flatten())
# fc6
model_E.add(Dense(4096, activation='relu'))
# fc7
model_E.add(Dense(4096, activation='relu'))
# fc8
model_E.add(Dense(num_classes, activation='softmax'))

model_E.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 56, 56, 256)      